### 测试

In [3]:
import os
import openpyxl
from openpyxl import Workbook
import numpy as np

def process_excel_files(root_folder, output_file="avg_std.xlsx"):
    # 创建输出工作簿
    output_wb = Workbook()
    output_ws = output_wb.active
    output_ws.title = "Averages and Standard Deviations"
    
    # 添加表头
    output_ws.append(["Filename", "Type", "Column1", "Column2", "Column3", "Column4", "Column5", "Column6"])
    
    # 遍历根文件夹下的所有子文件夹
    for foldername, subfolders, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith('.xlsx') and not filename.startswith('~$'):
                filepath = os.path.join(foldername, filename)
                file_basename = os.path.splitext(filename)[0]
                
                try:
                    # 打开Excel文件
                    wb = openpyxl.load_workbook(filepath, data_only=True)
                    ws = wb.active
                    
                    # 读取表头（假设在第一行）
                    headers = [cell.value for cell in ws[1]]
                    
                    # 初始化数据存储
                    data = [[] for _ in range(len(headers))]
                    
                    # 读取数据（假设从第2行到第41行，共40行）
                    for row in ws.iter_rows(min_row=2, max_row=41, values_only=True):
                        for i, value in enumerate(row):
                            if isinstance(value, (int, float)):
                                data[i].append(value)
                    
                    # 计算平均值和标准差
                    averages = [np.mean(col) if col else 0 for col in data]
                    # 平均值先扩大100倍，然后四舍五入保留两位小数
                    averages = [round(avg * 100, 2) for avg in averages]
                    std_devs = [np.std(col, ddof=1) if col else 0 for col in data]  # 使用样本标准差
                    # 标准差使用科学计数法计数，保留2位小数
                    std_devs = [f"{std:.2e}" for std in std_devs]
                    
                    # 写入平均值行
                    avg_row = [file_basename, "Average"] + averages
                    output_ws.append(avg_row)
                    
                    # 写入标准差行
                    std_row = [file_basename, "Standard Deviation"] + std_devs
                    output_ws.append(std_row)
                    
                except Exception as e:
                    print(f"Error processing file {filename}: {str(e)}")
    
    # 保存输出文件
    output_wb.save(root_folder+output_file)
    print(f"Results saved to {output_file}")

# 使用示例
if __name__ == "__main__":
    folder_path = "C:/Users/zsc/Desktop/Ablation study/Objective3/"
    process_excel_files(folder_path,output_file="obj_3_"+"avg_std.xlsx")

Results saved to obj_3_avg_std.xlsx
